In [1]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
nltk.download('stopwords')
from tqdm.auto import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maenz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from datasets import load_dataset
dataset = load_dataset("go_emotions", "raw")
df=pd.read_csv("my_emo_data")

Found cached dataset go_emotions (C:/Users/maenz/.cache/huggingface/datasets/go_emotions/raw/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
df_train=df.iloc[0:int(df.shape[0]*0.7)]
df_test=pd.DataFrame.from_dict(dataset['train'][1000:])
df_test=df_test.sample(frac=1).reset_index(drop=True)
df_test=df_test.iloc[3000:4000]

In [4]:
label_list=["admiration","amusement","anger","annoyance","approval","caring","confusion",\
            "curiosity","desire","disappointment","disapproval","disgust","embarrassment","excitement",\
            "fear","gratitude","grief","joy","love","nervousness","optimism","pride","realization","relief","remorse","sadness","surprise","neutral"]

In [5]:
positive=["admiration","amusement","approval","caring","curiosity","desire","excitement","gratitude","joy","love","optimism","pride","relief"]
negative=["anger","annoyance","confusion","disappointment","disapproval","disgust","embarrassment","fear","grief","nervousness","remorse",\
         "sadness"]

In [6]:
df_test["label"]=0

In [7]:
for index, row in df_test.iterrows():
    for i in range(len(label_list)):
        if row[label_list[i]]==1:
            if label_list[i] in positive:
                df_test.at[index,'label']=1
            elif label_list[i] in negative:
                df_test.at[index,'label']=0
            else:
                df_test.at[index,'label']=2

In [8]:
df_test=df_test[['text','label']]

In [ ]:
df_test=df_test.to_csv("hyper_select_valid")

In [ ]:
df_test=pd.read_csv("hyper_select_valid")

In [9]:
X_train_original=df_train['text']
X_valid_original=df_test['text']
y_train_original=df_train['label']
y_valid_original=df_test['label']
import torch
assert torch.cuda.is_available()
device = torch.device("cuda")

In [10]:
df_test

,text,label
3000,What a sad end for this franchise. I doubt Fal...,0
3001,"Oh, you had me worried there! Still if it does...",2
3002,Some people would be too ashamed to speak up t...,0
3003,Bullying impacts brain,0
3004,That sounds so terrible. I'm very sorry.,0
...,...,...
3995,Not super subtle with my water tool on top tho...,2
3996,Because punishing the poor was why they didn't...,0
3997,Wonder what the sub’s average age is.,2
3998,Pretending to be a racist just to troll people...,0


In [11]:
new_df=pd.read_csv("chatgpt_new_data_baseline")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])

In [12]:
from transformers import AutoTokenizer, AdamW,BertForSequenceClassification
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")
X_train=tokenizer(list(X_train_original),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_original)
X_valid=tokenizer(list(X_valid_original),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_valid=list(y_valid_original)

In [13]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, token):
        self.token = token
    def __getitem__(self, id):
        return {key: val[id] for key, val in self.token.items()}
    def __len__(self):
        return len(self.token.input_ids)

In [14]:
bat_size=128

In [15]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size,shuffle = False)

In [16]:
model_base = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model_base.to(device)
optimizer = AdamW(model_base.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model_base.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model_base.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model_base(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model_base.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model_base(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model_base.state_dict(), "my_model_base")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model_base.state_dict(), "my_model_base")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|          | 0/50 [00:00<?, ?it/s]

trainset:   0%|          | 0/6 [00:00<?, ?it/s]

validset:   0%|          | 0/8 [00:00<?, ?it/s]

epoch: 0
train loss: 1.1498114864031475
valid loss: 1.1070769280195236
valid accuracy: 0.397
——————————————————————————————————————————————


trainset:   0%|          | 0/6 [00:00<?, ?it/s]

...

validset:   0%|          | 0/8 [00:00<?, ?it/s]

epoch: 25
train loss: 0.7552293241024017
valid loss: 1.1884327232837677
valid accuracy: 0.435
——————————————————————————————————————————————


trainset:   0%|          | 0/6 [00:00<?, ?it/s]

validset:   0%|          | 0/8 [00:00<?, ?it/s]

early stop


In [18]:
print(max(valid_accuracy_list))

0.474
